In [9]:
import os
import pandas as pd

In [99]:
##Step Merge Together TADM data from Folder

files = ['Example/'+x for x in os.listdir('Example')]
files
curvefile = [x for x in files if 'Curves' in x][0]
curves_df = pd.read_csv(curvefile).set_index(['CurveID', 'Sheet'])
pressurevalues = {}
##Get unique pressurevalues
for pressurevalue in curves_df.index.unique(1):
    df = pd.read_csv([x for x in files if pressurevalue in x][0])
    df = df.set_index('Time').transpose()
    print(pressurevalue, df.columns[-1])
    df['Sheet'] = pressurevalue
    df.index.names = ['CurveID']
    df.reset_index(inplace=True)
    df['CurveID'] = df['CurveID'].astype(int)
    df.set_index(['CurveID','Sheet'], inplace=True)
    df = df.join(curves_df)
    pressurevalues[pressurevalue] = df
    

[x for x in pressurevalues['PressureValues11'].columns[-10:]]
tadm_data = pd.concat([pressurevalues[df] for df in pressurevalues],axis=0).set_index(['LiquidClassName','LiquidClassName',
                                                                            'Volume',
                                                                            'StepType',
                                                                            'Channel',
                                                                            'Time',
                                                                            'StepNumber',
                                                                            'TadmMode',
                                                                            'TadmError'],append=True)#.to_csv('mergedsheet.csv')

PressureValues0 3100
PressureValues1 4000
PressureValues2 6000
PressureValues3 4000
PressureValues4 6000
PressureValues5 450
PressureValues6 3600
PressureValues7 450
PressureValues8 3600
PressureValues9 8000
PressureValues10 8000
PressureValues11 47910
PressureValues12 31380
PressureValues13 3050
PressureValues14 3050
PressureValues15 640
PressureValues16 640
PressureValues17 2000
PressureValues18 3850
PressureValues19 640
PressureValues20 640
PressureValues21 2000
PressureValues22 3850
PressureValues23 750
PressureValues24 650
PressureValues25 650


In [94]:
##Step 2 Get Workflow Steps JSON Data
import json
import pandas as pd

##Define input File & Output File Directories
##Note Do not use "\" use only "/" in file name
inputFile = "Example/WorkflowSteps_20220718.json"
outputFile = "C:/Users/RipleyA/json_scrape_test4.csv"


##open json file
f = open (inputFile, "r")

##read each line of json file
Lines = f.readlines()

##create a list to store entries in
entries = []

##iterate through each line of json file
for line in Lines:
    
    ##create an empty entry object
    entry = {}
    
    ##get the json object from the line being read
    jsonobj = json.loads(line)
    
    ##iterate through each item in json object
    for item in range(0, len(jsonobj)):
         
        ##add each item in the json object to entry object
        entry[item] = jsonobj[item]['v']
    
    ##add filled in entry to the entries list
    entries.append(entry)
        
##create a dataframe from the entries
dataframe = pd.DataFrame(data=entries)

##perform type conversion of datetime columns 
dataframe[4] = dataframe[4].astype('datetime64')
dataframe[5] = dataframe[5].astype('datetime64')

##export dataframe to output file directory specified
dataframe

NameError: name 'json_data' is not defined

In [101]:
tadm_data.reset_index('Time')

,,,,,,,,,,Time,0,10,20,30,40,50,60,70,80,...,47820,47830,47840,47850,47860,47870,47880,47890,47900,47910
CurveID,Sheet,LiquidClassName,LiquidClassName,Volume,StepType,Channel,StepNumber,TadmMode,TadmError,,,,,,,,,,,,,,,,,,,,,
1,PressureValues0,NMDX_LHPA_Buffer_TADM,NMDX_LHPA_Buffer_TADM,700,Aspirating,1,717,0,0,7/18/2022 11:04:03 AM,34.0,34.0,39.0,39.0,39.0,39.0,39.0,39.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PressureValues0,NMDX_LHPA_Buffer_TADM,NMDX_LHPA_Buffer_TADM,700,Aspirating,2,717,0,0,7/18/2022 11:04:04 AM,39.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,-58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,PressureValues0,NMDX_LHPA_Buffer_TADM,NMDX_LHPA_Buffer_TADM,700,Aspirating,1,3846,0,0,7/19/2022 3:28:16 PM,29.0,29.0,29.0,29.0,29.0,29.0,34.0,34.0,-29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,PressureValues0,NMDX_LHPA_Buffer_TADM,NMDX_LHPA_Buffer_TADM,700,Aspirating,1,4087,0,0,7/19/2022 3:30:51 PM,19.0,19.0,14.0,19.0,19.0,19.0,19.0,19.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,PressureValues0,NMDX_LHPA_Buffer_TADM,NMDX_LHPA_Buffer_TADM,700,Aspirating,2,4087,0,0,7/19/2022 3:30:53 PM,29.0,29.0,29.0,29.0,29.0,34.0,29.0,34.0,-63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,PressureValues25,NeuMoDx_LHPC2_StandardVolumeFilter_Air_DispenseSurface,NeuMoDx_LHPC2_StandardVolumeFilter_Air_DispenseSurface,10,Aspirating,2,7205,0,0,7/21/2022 1:51:22 PM,-24.0,-24.0,-24.0,-24.0,-24.0,-24.0,-29.0,-24.0,-34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2308,PressureValues25,NeuMoDx_LHPC2_StandardVolumeFilter_Air_DispenseSurface,NeuMoDx_LHPC2_StandardVolumeFilter_Air_DispenseSurface,10,Aspirating,3,7205,0,0,7/21/2022 1:51:22 PM,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2309,PressureValues25,NeuMoDx_LHPC2_StandardVolumeFilter_Air_DispenseSurface,NeuMoDx_LHPC2_StandardVolumeFilter_Air_DispenseSurface,10,Aspirating,4,7205,0,0,7/21/2022 1:51:22 PM,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
dataframe[dataframe[1].str.contains('LhpA')]

,0,1,2,3,4,5
700,97bbf685-e233-4a96-a1f3-411ee6584e68,PerformLhpA,Gantry Events,Gantry Events,2022-07-18 16:03:42.467859400,2022-07-18 16:06:16.583728500
701,e8cd8930-b306-ed11-85f4-5cf370a20867,LhpAExecuting,CMV Plasma 3:VIR QUAL CMV Dual,,2022-07-18 16:03:42.263406300,2022-07-18 16:06:16.645563200
702,efcd8930-b306-ed11-85f4-5cf370a20867,LhpAExecuting,CMV Plasma 4:VIR QUAL CMV Dual,,2022-07-18 16:03:42.263406300,2022-07-18 16:06:16.645563200
704,e8cd8930-b306-ed11-85f4-5cf370a20867,LhpACompleted,CMV Plasma 3:VIR QUAL CMV Dual,,2022-07-18 16:06:16.646560900,2022-07-18 16:06:16.761253900
705,efcd8930-b306-ed11-85f4-5cf370a20867,LhpACompleted,CMV Plasma 4:VIR QUAL CMV Dual,,2022-07-18 16:06:16.646560900,2022-07-18 16:06:16.761253900
...,...,...,...,...,...,...
21837,f4bbce4d-3709-ed11-85f4-5cf370a20867,LhpAExecuting,BKV 10m4 U R8:VIR urine BKV 10,,2022-07-21 20:54:27.093383400,2022-07-21 20:56:02.454069700
21838,f4bbce4d-3709-ed11-85f4-5cf370a20867,LhpAExecuting,BKV 10m4 U R8:VIR urine BKV 10,,2022-07-21 20:56:02.454069700,2022-07-21 20:56:02.455066900
21847,97bbf685-e233-4a96-a1f3-411ee6584e68,PerformLhpA,Gantry Events,Gantry Events,2022-07-21 20:56:06.131853300,2022-07-21 20:57:41.434997500
21848,10592f87-3709-ed11-85f4-5cf370a20867,LhpAExecuting,BKV 10m4 U R9:VIR urine BKV 10,,2022-07-21 20:56:04.555070100,2022-07-21 20:57:41.467909500
